In [ ]:
# Analysis of the output of experiments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import datetime
import os
import pathlib

import pandas as pd
import altair as alt
import numpy as np
import pvlib

import plotly.express as px

alt.data_transformers.disable_max_rows()


def _(df, *args, **kwargs):
    print(len(df))
    display(df.head(*args, **kwargs))

In [ ]:
# It's always annoying to set the working directory: we use an environment variable defined in the Makefile.
CWD = os.environ.get("CWD")
if CWD:
    os.chdir(CWD)

In [ ]:
%pwd

In [ ]:
def round_to(x, to):
    return round(x / to) * to

In [ ]:
EXP_NAMES = [
    "model",
]

In [ ]:
COLORS = [
    "#086788",
    "#4c9a8e",
    "#ff9736",
    "#ffd053",
    #    "#63bcaf",
    #    "#e4e4e4",
    #    "#ffac5f",
    "#7bcdf3",
    "#14120e",
]


REMOVE_WEIRD_PV_IDS = False

# This rounding is only used in some charts.
ROUND_HORIZON_TO = 4

ROUND_PRED_TO = 60
NIGHT_THRESHOLD = 0.1

NORMALIZE_BY = "capacity"
METRIC = "mae"

In [ ]:
def load_exp(names):
    dfs = []
    for name in names:
        df = None
        for ext in [".csv.gz", ".csv"]:
            try:
                df = pd.read_csv(f"exp_results/{name}/test_errors{ext}")
            except FileNotFoundError as e:
                # print(e)
                continue

        if df is None:
            print(f"Could not find data for model {name}")
            continue
        df["model"] = name

        print(name)
        _(df)

        df = df[~df["y"].isna()]

        # Join the mean "y" per pv_id.
        mean_y = df[["y", "pv_id"]].groupby("pv_id").mean()  # )#df["y"].mean()
        mean_y.columns = ["mean_y"]
        df = df.join(mean_y, on="pv_id")

        # If a capacity is not already provided, define quantile(99%) as the capacity, per pv_id.
        if "capacity" not in df.columns:
            q99 = df[["y", "pv_id"]].groupby("pv_id").quantile(0.99)
            q99.columns = ["capacity"]
            df = df.join(q99, on="pv_id")

        dfs.append(df)

    df = pd.concat(dfs)

    return df


def augment_data(df):
    df["ts"] = pd.to_datetime(df["ts"])
    df = df[df["metric"] == "mae"]

    df["pred_ts"] = df["ts"] + pd.to_timedelta(df["horizon"], unit="minute")

    # We have heuristics to remove weird test PVs that have data at night.
    # In the `uk_pv` dataset there are quite a few of those.
    # Note that this is pretty much hard-coded for that use-case.
    if REMOVE_WEIRD_PV_IDS:
        night_mask = (df["pred_ts"].dt.hour < 1) | (df["pred_ts"].dt.hour > 20)
        bad_pvs = df[night_mask & df["y"] > 0]
        bad_pvs = bad_pvs[["pv_id", "y"]].groupby("pv_id").count().reset_index()
        bad_pvs = bad_pvs[bad_pvs["y"] > 10]
        # _(bad_pvs)

        remove_pvs = bad_pvs["pv_id"].unique()

        print(f"REMOVING PVS WITH NIGHT DATA: {remove_pvs}")
        df = df[~df["pv_id"].isin(remove_pvs)]

    if METRIC == "mae":
        pass
    elif METRIC == "mbe":
        # For mean bias error, recompute the errors as we don't compute that one by default.
        df["error"] = df["y"] - df["pred"]

    # df = df.join(inferred_meta[["factor", "capacity"]], on="pv_id")
    # df = df.join(meta[['latitude', 'longitude']], on="pv_id")

    # Normalizing by the average `y` values for each PV.
    if NORMALIZE_BY == "mean":
        df["weighted_error"] = df["error"] / df["mean_y"] * 100.0
    elif NORMALIZE_BY == "capacity":
        df["weighted_error"] = df["error"] / df["capacity"] * 100.0
    elif NORMALIZE_BY == "nothing":
        df["weighted_error"] = df["error"]
    else:
        raise ValueError("unknown NORMALIZE_BY value")

    df["horizon"] = df["horizon"] / 60.0
    df = df[~df["error"].isnull()]

    # Round the prediction hour and the horizon for more concise charts.
    df["pred_hour"] = df["pred_ts"].dt.hour * 60 + round_to(df["pred_ts"].dt.minute, ROUND_PRED_TO)

    df = df[df["y"] > NIGHT_THRESHOLD]

    return df

In [ ]:
error_name = METRIC.upper()
if NORMALIZE_BY == "mean":
    error_name += " / avg  (%)"
elif NORMALIZE_BY == "capacity":
    error_name += " / capacity (%)"
elif NORMALIZE_BY == "nothing":
    error_name += " (MW)"
else:
    raise ValueError("unknown NORMALIZE_BY value")

In [ ]:
df = load_exp(EXP_NAMES)
_(df, 20)
# _(df.sample(20), 20)
df = augment_data(df)
_(df, 20)
# _(df.sample(20), 20)

In [ ]:
df["model"] = pd.Categorical(df["model"], reversed(EXP_NAMES))
df = df.sort_values("model")

In [ ]:
def print_means(df):
    mean_ = df.groupby(["model"])["weighted_error"].mean().to_dict()
    err = (
        df.groupby(["model"])["weighted_error"].std()
        * 1.96
        / np.sqrt(df.groupby(["model"])["weighted_error"].count())
    ).to_dict()

    print("Mean: ", error_name)
    for name in mean_:
        print(f"{name}: {mean_[name]:.2f} ± {err[name]:.2f}")

In [ ]:
print_means(df)

In [ ]:
color_scale = alt.Scale(range=COLORS)

In [ ]:
gt = df[["y", "pred_hour"]].copy()
gt = gt.groupby(["pred_hour"]).mean().reset_index()
gt["pred_hour"] = pd.to_timedelta(gt["pred_hour"], unit="minute")
gt["date"] = pd.Timestamp(2023, 1, 1)
gt["pred_hour"] = gt["date"] + gt["pred_hour"]
gt["y"] /= 10
del gt["date"]

# _(df)
data = df[["model", "horizon", "weighted_error", "pred_hour"]].copy()
# data = data[(data['horizon'] % 4).isin([0, 1, 3])]
data["horizon"] = (data["horizon"] // 4) * 4
# data['pred_hour'] = round(data['pred_hour'] / 60) * 60
# _(data)
data = data.groupby(["model", "horizon", "pred_hour"]).agg(["mean", "std", "count"]).reset_index()
data["error"] = data[("weighted_error", "mean")]
err = 1.96 * data[("weighted_error", "std")] / data[("weighted_error", "count")].pow(0.5)
data["low"] = data[("weighted_error", "mean")] - err
data["high"] = data[("weighted_error", "mean")] + err
del data["weighted_error"]
data.columns = data.columns.get_level_values(0)
# _(data)

data["pred_hour"] = pd.to_timedelta(data["pred_hour"], unit="minute")
data["date"] = pd.Timestamp(2023, 1, 1)
data["pred_hour"] = data["date"] + data["pred_hour"]

# del data['pred_hour']
del data["date"]
data = data.sort_values(["model", "pred_hour", "horizon"])

h = data["horizon"].astype(int)
data["h_label"] = "[ " + h.astype(str) + ", " + (h + 4).astype(str) + " ["
_(data)

c = alt.Chart()

line = c.mark_line(point=True).encode(
    x=alt.X("hoursminutes(pred_hour)", title="Prediction time of day"),
    y=alt.Y("error", title=error_name),
    color=alt.Color("model:N", title="Model", sort=EXP_NAMES, scale=color_scale),
    # facet=alt.Facet("horizon:O", title='Horizon', spacing=10, columns=5),
)

error = c.mark_errorband().encode(
    x=alt.X("hoursminutes(pred_hour)", title=""),
    y=alt.Y("high", title=""),
    y2=alt.Y2("low", title=""),
    color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
)

# gt_chart = (
#     alt.Chart(gt)
#     .mark_line(color='black')
#     .encode(
#         x='hoursminutes(pred_hour)',
#         y='y',
#     )
# )

c = (
    alt.layer(line, error, data=data)
    .properties(
        height=80,
        width=140,
    )
    .facet(alt.Facet("h_label", title="Horizon", sort=alt.EncodingSortField("horizon")), columns=4)
    .configure_point(size=0)
)
c

In [ ]:
data = df[df["model"] == EXP_NAMES[0]].copy()
data = data[["horizon", "weighted_error", "pred_hour"]]

data["horizon"] = (data["horizon"] // 4) * 4

data = data.groupby(["horizon", "pred_hour"]).agg(["mean", "std", "count"]).reset_index()

data["error"] = data[("weighted_error", "mean")]
err = 1.96 * data[("weighted_error", "std")] / data[("weighted_error", "count")].pow(0.5)
data["low"] = data[("weighted_error", "mean")] - err
data["high"] = data[("weighted_error", "mean")] + err
del data["weighted_error"]
data.columns = data.columns.get_level_values(0)

data["pred_hour"] = pd.to_timedelta(data["pred_hour"], unit="minute")
data["date"] = pd.Timestamp(2023, 1, 1)
data["pred_hour"] = data["date"] + data["pred_hour"]

del data["date"]

h = data["horizon"].astype(int)
data["h_label"] = "[ " + h.astype(str) + ", " + (h + 4).astype(str) + " ["
_(data)

c = alt.Chart()

line = c.mark_line(point=True).encode(
    x=alt.X("hoursminutes(pred_hour)", title="Prediction time of day"),
    y=alt.Y("error", title=error_name),
    color=alt.Color("horizon"),
)

error = c.mark_errorband().encode(
    x=alt.X("hoursminutes(pred_hour)", title=""),
    y=alt.Y("high", title=""),
    y2=alt.Y2("low", title=""),
    color=alt.Color("horizon"),
)

# gt_chart = (
#     alt.Chart(gt)
#     .mark_line(color='black')
#     .encode(
#         x='hoursminutes(pred_hour)',
#         y='y',
#     )
# )

c = (
    alt.layer(line, data=data)
    .properties(
        height=200,
        width=500,
    )
    .configure_point(size=0)
)
c

In [ ]:
data = df[["model", "horizon", "weighted_error"]].copy()
data["horizon"] = round_to(data["horizon"], 1)
data = data.groupby(["model", "horizon"]).agg(["mean", "std", "count"]).reset_index()

data["error"] = data[("weighted_error", "mean")]
err = 1.96 * data[("weighted_error", "std")] / data[("weighted_error", "count")].pow(0.5)
data["high"] = data["error"] + err
data["low"] = data["error"] - err

data.columns = data.columns.get_level_values(0)

_(data)


line = (
    alt.Chart(data[["model", "error", "horizon"]])
    .mark_line(interpolate="step-after", point=True)
    .encode(
        y=alt.Y("error", title=error_name, scale=alt.Scale(zero=False)),
        color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
        x=alt.X("horizon:O", title="Horizon"),
    )
)

error = (
    alt.Chart(data[["model", "horizon", "high", "low"]])
    .mark_errorband(interpolate="step-after", opacity=0.15)
    .encode(
        x="horizon:O",
        y2="low",
        y=alt.Y("high", title=""),
        color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
    )
)

c = (
    alt.layer(line, error).properties(
        height=250,
        width=700,
    )
    # hack to get the opacity=1 legend without the points!
    .configure_point(size=0)
)

c

In [ ]:
print(aggregated_data)
alt.Chart(aggregated_data).mark_line(point=True).encode(
    x=alt.X("year_month:N"), y=alt.Y("mean_error")
).properties(height=200, width=700)

In [ ]:
data = df[["model", "pred_ts", "weighted_error"]].copy()

data["year_month"] = data["pred_ts"].dt.strftime("%Y-%m")
del data["pred_ts"]
data = data
data = data.groupby(["model", "year_month"]).agg(["mean", "std", "count"]).reset_index()

data["error"] = data[("weighted_error", "mean")]
err = 1.96 * data[("weighted_error", "std")] / data[("weighted_error", "count")].pow(0.5)
data["high"] = data["error"] + err
data["low"] = data["error"] - err

data.columns = data.columns.get_level_values(0)

del data["weighted_error"]

_(data)
data.to_csv("patate.csv")

line = (
    alt.Chart(data[["error", "year_month", "model"]])
    .mark_line(point=True)
    .encode(
        x=alt.X("year_month:N", title=None, axis=alt.Axis(labelAngle=-45, labelFontSize=14)),
        y=alt.Y("error", title=error_name, scale=alt.Scale(zero=True)),
        color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
    )
)

error = (
    alt.Chart(data[["model", "year_month", "low", "high"]])
    .mark_errorband(opacity=0.2)
    .encode(
        x="year_month",
        y=alt.Y("low", title=""),
        y2="high",
        color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
    )
)


(line + error).properties(height=200, width=700)

In [ ]:
# Same but per PV id.

data = df[["model", "pred_ts", "weighted_error", "pv_id"]].copy()

data["year_month"] = data["pred_ts"].dt.strftime("%Y-%m")
del data["pred_ts"]
data = data
data = data.groupby(["model", "year_month", "pv_id"]).agg(["mean", "std", "count"]).reset_index()

data["error"] = data[("weighted_error", "mean")]
err = 1.96 * data[("weighted_error", "std")] / data[("weighted_error", "count")].pow(0.5)
data["high"] = data["error"] + err
data["low"] = data["error"] - err

data.columns = data.columns.get_level_values(0)

del data["weighted_error"]

_(data)
# data.to_csv("patate.csv")

line = (
    alt.Chart()
    .mark_line(point=True)
    .encode(
        x=alt.X("year_month:N", title=None, axis=alt.Axis(labelAngle=-45, labelFontSize=14)),
        y=alt.Y("error", title=error_name, scale=alt.Scale(zero=True)),
        color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
    )
)

error = (
    alt.Chart()
    .mark_errorband(opacity=0.2)
    .encode(
        x="year_month",
        y=alt.Y("low", title=""),
        y2="high",
        color=alt.Color("model", sort=EXP_NAMES, scale=color_scale),
    )
)

(
    alt.layer(line, error, data=data)
    .properties(height=100, width=300)
    .facet(alt.Facet("pv_id"), columns=2)
)

In [ ]:
_(df)

scale = alt.Scale()

max_ = df["y"].quantile(0.99)

data = df.sample(10000)

points = (
    alt.Chart()
    .mark_circle(opacity=0.8)
    .encode(
        x=alt.X("y", scale=scale, title="ground truth"),
        y=alt.Y("pred", scale=scale, title="Prediction"),
        color=alt.Color("model", scale=color_scale, sort=EXP_NAMES),
    )
    # .properties(width=400, height=400)
)

line = (
    alt.Chart(pd.DataFrame(dict(x=[0, max_], y=[0, max_])))
    .mark_line(color="black", size=1)
    .encode(x="x", y="y")
)

(
    alt.layer(points, line, data=data)
    .properties(width=200, height=200)
    .facet("model", columns=np.ceil(np.sqrt(len(EXP_NAMES))))
)

In [ ]:
# Error distribution

data = df[["y", "pred", "mean_y", "model"]].copy()
data["error"] = (df["pred"] - df["y"]) / df["mean_y"]
_(data)

scale = alt.Scale()  # type='log')
# df = df[df['y'] > 0]
# df = df[df['pred'] > 0]

chart = (
    alt.Chart(data.sample(10000))
    .encode(
        x=alt.X("error", scale=scale, title="Error", bin=alt.Bin(maxbins=100)),
        y=alt.Y("count()", stack=False),
        color=alt.Color("model", scale=color_scale, sort=EXP_NAMES),
    )
    .properties(width=700, height=400)
)


line = alt.Chart(pd.DataFrame(dict(x=[0]))).mark_rule().encode(x="x")

(chart.mark_line(point=True) + chart.mark_area(opacity=0.1) + line).configure_point(size=0)